# Homework Assignment 6 - Evan Callaghan

### 1. A data scientist is running an AdaBoost classifier on a dataset with 100 observations. Answer the following:

### a) What is the weight initial weight of observation 72th in the training dataset?

#### Initially, an AdaBoost classifier weighs all observations equally. Since there is 100 observations under consideration, the initial weight of the 72nd observation is 1/100 (0.01). 

### b) The 72nd observation in the training dataset is misclassified by the first weak learner chosen by the data scientist. Is the new weight of the 72nd observation in the training dataset larger or smaller than the weight assigned to that observation initially?

#### AdaBoost works to increase the weight of misclassified observations in order to force the learner to focus on the regions that were more problematic for the previous estimators. Therefore, since the 72nd observation is misclassified by the first learner, the new weight will be larger than the weight initially assigned. 

### 2. Explain why AbaBoost.M1 is an ensemble learning algorithm?

#### An ensemble learning algorithm is defined as a set of weak learners that are trained together (or in a sequence) to make up a committee. In classification and regression problems, the final result is obtained by averaging the predictions or employing a majority vote. In AdaBoost.M1, a set number of weak models are added sequentially and trained using the weighted training data. Therefore, AdaBoost.M1 is an ensemble learning algorithm.


### 3. Suppose you are running AdaBoost.M1 (with η = 0.1) with 4 training examples. At the start of the current iteration, the four examples have the weights shown in the following table. Another column says if the weak classifier got them correct or incorrect. Determining the new weights for these four examples:

#### w(t) = [0.16, 0.64, 0.08, 0.12], eta = 0.1, o1 = -1, o2 = -1, o3 = 1, o4 = 1,
#### E(t) = (0.08 + 0.12) / (0.16 + 0.64 + 0.08 + 0.12) = 0.2
#### alpha(t) = log((1 - E(t)) / E(t)) = log((1 - 0.2) / 0.2) = 0.60206

#### w1(t+1) = w1(t)e^(eta * alpha(t) * o1) = 0.16e^(0.1 * 0.60206 * -1) = 0.15065
#### w2(t+1) = w2(t)e^(eta * alpha(t) * o2) = 0.64e^(0.1 * 0.60206 * -1) = 0.60261
#### w3(t+1) = w3(t)e^(eta * alpha(t) * o3) = 0.08e^(0.1 * 0.60206 * 1) = 0.08496
#### w4(t+1) = w4(t)e^(eta * alpha(t) * o4) = 0.12e^(0.1 * 0.60206 * 1) = 0.12745

#### Normalized: w(t+1) = [0.156, 0.624, 0.088, 0.132]

### 4.  If your AdaBoost ensemble under-fits the training dataset, what would you do to fix that? That is, which hyper-parameters should you tweak?

#### If your AdaBoost ensemble under-fits the training dataset, I would try to increase the number of estimators or reduce the regularization hyper-parameter of the base estimator. These options could increase the complexity of the model and avoid the problem of under-fitting.


### 5. For binary classification, which of the following statements are TRUE of AdaBoost with decision trees as learners?

#### A. It usually has lower bias than a single decision tree.
#### B. It is popular because it usually works well even before any hyper-parameter tuning 
#### C. It assigns higher weights to observations that have been misclassified.

In [1]:
## 6. a) Using the pandas library to read the csv data file and create a data-frame called heart

import boto3
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import recall_score

## Defining the bucket
s3 = boto3.resource('s3')
bucket_name = 'data-445-bucket-callaghan'
bucket = s3.Bucket(bucket_name)

## Defining the csv file
file_key = 'framingham.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

heart = pd.read_csv(file_content_stream)

## Removing observation with missing values
heart = heart.dropna()

heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [2]:
## b) Using age, totChol, sysBP,BMI, heartRate, and glucose as the predictor variables and TenYearCHD as the target 
## variable to do the following:

## Defining the input and target variables:
X = heart[['age', 'totChol', 'sysBP', 'BMI', 'heartRate', 'glucose']]
Y = heart['TenYearCHD']


## Defining empty lists to store results
md1_recall = []
md2_recall = []
md3_recall = []


## Repeating the steps 100 times
for i in range(0, 100):

    ## Splitting the data
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y)

    ## Normalizing the input variables
    scaler = MinMaxScaler(feature_range = (0,1))
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.fit_transform(X_test)
    
    
    ## Model 1
    ## Building a random forest classifier (using 500 trees and maximum depth tree equal to 3)
    ## ------------------------------------------

    ## Building the model
    md1 = RandomForestClassifier(n_estimators = 500, max_depth = 3).fit(X_train, Y_train)

    ## Predicting on the test set
    md1_preds = md1.predict_proba(X_test)[:, 1]

    ## Using 10% as cutoff value and reporting the recall
    md1_preds = np.where(md1_preds < 0.1, 0, 1)
    md1_recall.append(recall_score(Y_test, md1_preds))



    ## Model 2
    ## Building an AdaBoost classifier (using 500 trees and maximum depth tree equal to 3)
    ## ------------------------------------------

    ## Building the model
    md2 = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth = 3), n_estimators = 500).fit(X_train, Y_train)

    ## Predicting on the test set
    md2_preds = md2.predict_proba(X_test)[:, 1]

    ## Using 10% as cutoff value and reporting the recall
    md2_preds = np.where(md2_preds < 0.1, 0, 1)
    md2_recall.append(recall_score(Y_test, md2_preds))



    ## Model 3
    ## Building an AdaBoost classifier (using 50 learners and the support vector machine classifier 
    ## as the learner with rbf as the kernel)
    ## ------------------------------------------

    ## Building the model
    md3 = AdaBoostClassifier(base_estimator = SVC(kernel = 'rbf', probability = True), n_estimators = 50).fit(X_train, Y_train)

    ## Predicting on the test set
    md3_preds = md3.predict_proba(X_test)[:, 1]

    ## Using 10% as cutoff value and reporting the recall
    md3_preds = np.where(md3_preds < 0.1, 0, 1)
    md3_recall.append(recall_score(Y_test, md3_preds))
    print(i)

## Computing the average recall of each of the models across the 100 iterations
print('Average Recall Score of Model 1:', np.mean(md1_recall))
print('Average Recall Score of Model 2:', np.mean(md2_recall))
print('Average Recall Score of Model 3:', np.mean(md3_recall))

## I would use either the AdaBoost model with a decision tree or the AdaBoost model with
## a support vector machine to predict TenYearCHD because they have the two highest
## recall scores of the three considered models

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
Average Recall Score of Model 1: 0.9232142857142855
Average Recall Score of Model 2: 1.0
Average Recall Score of Model 3: 1.0
